# Klassifikation

In diesem Beispiel versuchen wir ein tieferes Verständnis für Logistische Regression und Softmax Regression zu bekommen. Dazu plotten wir einige Entscheidungsgrenzen unserer Klassifikatoren.

Zuerst laden wir das Iris Datenset. Dieses Datenset beinhaltet 150 Beobachtungen von jeweils vier Attributen von Schwertlilien. Gemessen wurden dabei jeweils die Breite und die Länge des Kelchblatts (Sepalum) sowie des Kronblatts (Petalum) in Zentimeter. Des weiteren ist für jeden Datensatz die Art der Schwertlilie (Iris setosa, Iris virginica oder Iris versicolor) angegeben.

In [20]:
from sklearn.datasets import load_iris
data = load_iris(as_frame=True)
data.data.head()

In [21]:
data.target_names

## Binärer Klassifikator

Zu Visualisierungszwecken benutzen wir nur die Features Petal-Länge (Index 2) und Petal-Breite (Index 3). Weiters wollen wir zuerst einen Klassifikator trainieren, welcher Iris Virginica (Index 2) von den restlichen Iris Arten unterscheiden kann.

In [22]:
from sklearn.model_selection import train_test_split
X = data.data.iloc[:, 2:]  # wähle nur `petal length` und `petal width`
y = data.target == 2  # Binärer Vektor, 1: Iris Virginica, 0: nicht Iris Virginica
X.head()

## 3a. Binäre Logistische Regression
- Trainiere eine binäre Logistische Regression (`sklearn.linear_model.LogisticRegression`) um Iris Virginica zu erkennen.
- Benutze eine l2 Regularisierung `penalty='l2'` und `GridSearchCV` für den Parameter `C`.
- Welchen Einfluss hat der Hyperparameter C?

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from sklearn.model_selection import GridSearchCV

In [24]:
log_reg = LogisticRegression(penalty='l2')
log_reg.fit(X_train, y_train)
cross_validate(log_reg, X, y, cv = 3)
C = np.logspace(0, 4, 10)
hyperparameters = dict(C=C)
             
grid_search=GridSearchCV(log_reg,hyperparameters, cv = 5, verbose = 1, n_jobs= 1)
best_model=grid_search.fit(X_train, y_train)
print(best_model.best_estimator_.get_params()['C'])
print("The mean accuracy of the model is:",best_model.score(X_train,y_train))

Im nächsten Schritt wollen wir die Entscheidungsgrenze plotten. Forme dazu die Klassenzugehörigkeitswahrscheinlichkeit in Form der Sigmoid Funktion auf $x_2$ (Petal-Breite) um und setze $p=0.5$.
$$ p = \sigma(x) = {1 \over 1 + \exp(\beta_0 + \beta_1 x_1 + \beta_2 x_2)}$$

- Schreibe eine Python Funktion (benutze die Parameter `intercept_` und `coef_`).
- Erstelle einen Plot mit dem Datensatz (nur Petal Länge und Petal Breite) und zeichne die Entscheidungsgrenze ein.


In [25]:
import numpy as np
import matplotlib.pyplot as plt
log_reg_n = LogisticRegression()
log_reg_n.fit(X_train, y_train)

def decision_boundary(x):
    b0 = log_reg_n.intercept_[0]
    b1,b2 = log_reg_n.coef_[0]
    x2 = -(b0+b1*x)/b2
    return x2

plt.scatter(X_train.iloc[:, 0], X_train.iloc[:, 1], c=y_train)
plt.xlabel('petal length')
plt.ylabel('petal width')
x = np.arange(2,7,0.1)
y = decision_boundary(x)
plt.plot(x, y)


## 3b. Entscheidungsgrenze von Iris Versicolor
- Trainiere eine Logistische Regression für Iris Versicolor.
- Plotte wie im vorherigen Beispiel die Entscheidungsgrenze.

In [27]:
X = data.data.iloc[:, 2:]  # wähle nur `petal length` und `petal width`
y = data.target == 1  # Binärer Vektor 1: Iris Versicolor, 0: nicht Iris Versicolor
X_train, X_test, y_train, y_test = train_test_split(X, y == 1, test_size=0.2, random_state=42)

In [28]:
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

def decision_boundary(x):
    b0 = log_reg.intercept_[0]
    b1,b2 = log_reg.coef_[0]
    x2 = -(b0+b1*x)/b2
    return x2

plt.scatter(X_train.iloc[:, 0], X_train.iloc[:, 1], c=y_train)
plt.xlabel("petal length Iris Versicolor")
plt.ylabel("petal with Iris Versicolor")
x=np.arange(2,7,0.1)
y=decision_boundary(x)
plt.plot(x, y)


## 3c. Softmax Regression
Um einen Multioutput Klassifikator zu trainieren, können wir einfach den ursprünglichen Targetvektor verwenden. `sklearn` trainiert dann eine Softmax Regression.


In [14]:
X = data.data.iloc[:, 2:]  # wähle nur `petal length` und `petal width`
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

model = LogisticRegression(penalty='none', multi_class='multinomial')
model.fit(X_train, y_train)

Um die Entschdeidungsgrenzen zu plotten verwenden wir einen Konturplot.

In [34]:
import numpy as np
import matplotlib.pyplot as plt

def plot_decision_boundary(model, X, y):
    plt.figure()
    petal_length = np.arange(1, 7, 0.1)
    petal_width = np.arange(0, 3, 0.1)

    X_grid, Y_grid = np.meshgrid(petal_length, petal_width)

    def pred(x, y, label):
        return model.predict_proba(np.c_[x, y])[0, label]

    pred = np.frompyfunc(pred, 3, 1, )

    plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=y)

    Z = pred(X_grid, Y_grid, 0)
    plt.contour(X_grid, Y_grid, Z, [0.5])
    Z = pred(X_grid, Y_grid, 1)
    plt.contour(X_grid, Y_grid, Z, [0.5])
    Z = pred(X_grid, Y_grid, 2)
    plt.contour(X_grid, Y_grid, Z, [0.5])
    
plot_decision_boundary(model, X_train, y_train)

- Plotte die Entscheidungsgrenzen für verschiedene Werte des Hyperparameters `C`. Verwende dazu `penalty='l2'`.
- Berechne die Wahrheitsmatrix (Confusion Matrix)
- Berechne folgenden Werte: `accuracy_score, precision_score, recall_score, f1_score`

In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_pred)

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
